In [1]:
import tkinter as tk
from PIL import Image, ImageTk
import pandas as pd
import random  # For assigning random statuses

In [2]:
# Create seat data
def create_seat_data(rows=5):
    seat_data = []
    positions = ['window', 'middle', 'aisle', 'aisle', 'middle', 'window']
    statuses = ['available', 'occupied', 'ghost']
    for row_label in "ABCDEF"[:rows]:
        for col in range(1, 7):
            seat_id = f"{row_label}{col}"
            position = positions[col - 1]
            status = random.choice(statuses)
            if row_label == "A":
                category = "business"
                price = 200 if position == 'window' else 150 if position == 'aisle' else 120
            else:
                category = "economy"
                price = 100 if position == 'window' else 80 if position == 'aisle' else 70
            seat_data.append({'Seat ID': seat_id, 'Status': status, 'Category': category, 'Position': position, 'Price': price})
    return pd.DataFrame(seat_data)

# Load images
def load_images():
    base_path = r"C:\Users\asus.LAPTOP-SMDU8F43\Desktop\FRANS Airlines\images"
    positions = ['left_window', 'middle', 'right_window', 'left_aisle', 'right_aisle']
    statuses = ['available', 'occupied', 'ghost']
    images = {}
    for status in statuses:
        for position in positions:
            key = f"{status}_{position}"
            img_path = f"{base_path}\\{key}.png"
            try:
                images[key] = ImageTk.PhotoImage(Image.open(img_path).resize((60, 60)))
            except Exception as e:
                print(f"Error loading image: {img_path}. Exception: {e}")
    return images

# Display seats with images and text
def display_seats(dataframe):
    root = tk.Tk()
    root.title("Seat Reservation System")
    images = load_images()

    for index, row in dataframe.iterrows():
        # Adjust positions for left and right visuals
        if row['Position'] == 'window' and index % 6 < 3:
            position = 'left_window'
        elif row['Position'] == 'window':
            position = 'right_window'
        elif row['Position'] == 'aisle' and index % 6 < 3:
            position = 'left_aisle'
        elif row['Position'] == 'aisle':
            position = 'right_aisle'
        else:
            position = row['Position']

        key = f"{row['Status']}_{position}"
        image = images.get(key, None)

        # Determine text to display
        if row['Status'] == 'available':
            display_text = row['Seat ID']
        elif row['Status'] == 'occupied':
            display_text = "X"
        else:
            display_text = "*"

        col = index % 6
        grid_col = col if col < 3 else col + 1
        grid_row = index // 6

        # Create seat button with image or fallback to text
        if image:
            seat = tk.Button(root, image=image, text=display_text, compound="center",
                             font=("Arial", 12, "bold"),
                             command=lambda seat_id=row['Seat ID']: reserve_seat(seat_id, dataframe, root))
            seat.image = image
        else:
            seat = tk.Button(root, text=display_text, width=8, height=3,
                             command=lambda seat_id=row['Seat ID']: reserve_seat(seat_id, dataframe, root))
        seat.grid(row=grid_row, column=grid_col, padx=10, pady=10)

    for r in range(len(dataframe) // 6):
        tk.Label(root, text="|", font=("Arial", 14)).grid(row=r, column=3, padx=10, pady=10)

    root.mainloop()

# Reserve seat
def reserve_seat(seat_id, dataframe, root):
    current_status = dataframe.loc[dataframe['Seat ID'] == seat_id, 'Status'].values[0]
    if current_status == 'available':
        dataframe.loc[dataframe['Seat ID'] == seat_id, 'Status'] = 'occupied'
        display_seats(dataframe)
    elif current_status == 'occupied':
        print(f"Seat {seat_id} is already occupied!")
    elif current_status == 'ghost':
        print(f"Seat {seat_id} is temporarily selected (ghost).")

# Run the application
seats_df = create_seat_data(rows=5)
display_seats(seats_df)